In [ ]:
#####################################################################
#
#       aiplatform.CustomTrainingJob -> script_path
#
#####################################################################

In [ ]:
#####################################################################
#
# define the training script
#
#####################################################################

In [44]:
%%writefile training_script.py

import argparse
import pickle
import os

# Sample Decision Tree Classifier
from sklearn import datasets
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

from google.cloud import storage

# parse args
parser = argparse.ArgumentParser()
parser.add_argument('--bucket_name', dest='bucket_name', default="", type=str, help='The GCS bucket to store model artifacts -> w/o gs://')
parser.add_argument('--max_depth', dest='max_depth', default=10, type=int, help='The maximum depth of the tree')
args = parser.parse_args()

# load the iris datasets
dataset = datasets.load_iris()

# fit a CART model to the data
model = DecisionTreeClassifier(max_depth = args.max_depth)
model.fit(dataset.data, dataset.target)
print(model)

# make predictions
expected = dataset.target
predicted = model.predict(dataset.data)

# summarize the fit of the model
classification_report = metrics.classification_report(expected, predicted)
confusion_matrix = metrics.confusion_matrix(expected, predicted)

# save the model to disk
model_filename = "model.pkl"
with open(model_filename, 'wb') as model_file:
    pickle.dump(model, model_file)

# Upload the saved model file to GCS
storage_client = storage.Client()
bucket = storage_client.get_bucket(args.bucket_name)
model_directory = os.environ["AIP_MODEL_DIR"]
storage_path = os.path.join(model_directory, model_filename)
blob = storage.blob.Blob.from_string(storage_path, client=storage_client)
blob.upload_from_filename(model_filename)

Overwriting training_script.py


In [45]:
#####################################################################
#
# kick off the custom training job
#
#####################################################################

In [46]:
from google.cloud import aiplatform
from datetime import datetime

In [47]:
# specify parameters
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]
REGION = "us-central1"
BUCKET_NAME = f"bkt-{PROJECT_ID}-vpipelines"
BUCKET_PATH = f"gs://{BUCKET_NAME}"
PIPELINE_ROOT = f"{BUCKET_PATH}/pipeline_root"
PIPELINE_DATA = f"{BUCKET_PATH}/data"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [48]:
TRAIN_VERSION  = "scikit-learn-cpu.0-23"
DEPLOY_VERSION = "sklearn-cpu.0-23"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

JOB_NAME = "vai_CustomTrainingJob"

MAX_DEPTH = 20
CMDARGS = [  f"--bucket_name={BUCKET_NAME}"
           , f"--max_depth={str(MAX_DEPTH)}"
          ]

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_PATH)

job = aiplatform.CustomTrainingJob(
    display_name = JOB_NAME
    , script_path = "training_script.py"
    , container_uri = TRAIN_IMAGE
    #, requirements = ["tensorflow_datasets"]
    , model_serving_container_image_uri = DEPLOY_IMAGE
)

MODEL_DISPLAY_NAME = "vai_model_registry_name"

# Start the training
model = job.run(
    model_display_name = MODEL_DISPLAY_NAME
    , args = CMDARGS
    , replica_count = 1
    , service_account = f"sa-vertex-pipelines@{PROJECT_ID}.iam.gserviceaccount.com"
)

Training script copied to:
gs://bkt-ap-alto-ml-1000-vpipelines/aiplatform-2022-12-19-23:47:59.948-aiplatform_custom_trainer_script-0.1.tar.gz.
Training Output directory:
gs://bkt-ap-alto-ml-1000-vpipelines/aiplatform-custom-training-2022-12-19-23:48:00.110 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2596916859873787904?project=720376660491
CustomTrainingJob projects/720376660491/locations/us-central1/trainingPipelines/2596916859873787904 current state:
PipelineState.PIPELINE_STATE_RUNNING
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2906565722534576128?project=720376660491
CustomTrainingJob projects/720376660491/locations/us-central1/trainingPipelines/2596916859873787904 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/720376660491/locations/us-central1/trainingPipelines/2596916859873787904 current state:
PipelineState.PIPELINE_STATE_RUNNING
